In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
df1 = pd.read_csv("../input/university-salaries/university-salaries/salaries_final.csv")
df2 = pd.read_csv("../input/university-salaries/university-salaries/salaries_without_dept.csv")
df3 = pd.read_csv("../input/university-salaries/university-salaries/data_dictionary.csv")

In [4]:
df1

In [5]:
df1_year = df1.groupby(['Year'])
df1_year.nunique()

In [6]:
df1_name = df1.groupby(['Name'])
df1_name.nunique()

In [7]:
df1_job = df1.groupby(['Primary Job Title'])
df1_job.nunique()

In [8]:
df1_dept = df1.groupby(['Department'])
df1_dept.nunique()

In [9]:
df1_coll = df1.groupby(['College'])
df1_coll.nunique()

In [28]:
df1['Primary Job Title'].value_counts().head(20)

ดูการเพิ่มขึ้นของตำแหน่งงาน 5 อันดับแรก

In [11]:
df1_job = df1.loc[df1['Primary Job Title'].isin(['Assistant Professor', 'Associate Professor', 'Professor', 'Lecturer', 'Senior Lecturer'])]
fig, ax = plt.subplots(1, 5, figsize = (20, 5), sharex = True)
job = ['Assistant Professor', 'Associate Professor', 'Professor', 'Lecturer', 'Senior Lecturer']
for i in range(0,5):
    plt.sca(ax[i])
    df1_job.loc[df1_job['Primary Job Title'] == job[i], 'Year'].value_counts().sort_values().plot(kind = 'bar')
    plt.title(job[i], fontsize = 16)
    plt.xlabel("Years")
plt.tight_layout
plt.show()

จากกราฟข้างบน หาอัตราการเติบโตเพิ่ม 

In [12]:
df3['College Abbreviation']

In [13]:
df1.groupby(['College', 'Year'])[['Base Pay']].mean()

In [26]:
coll = df3['College Abbreviation']
for i in range(0,12):
    df1_coll_year = df1.loc[df1['College'] == coll[i]].groupby(['College', 'Year'])[['Base Pay']].mean().sort_values(by = 'Year').plot(kind = 'bar').plot()
    plt.title(coll[i], fontsize = 16)
    plt.ylabel("Average Base Pay")
plt.tight_layout
plt.show()

สรุปผลจากกราฟข้างต้น

In [ ]:
coll = df3['College Abbreviation']
for i in range(0,12):
    df1_coll_year = df1.loc[df1['College'] == coll[i]].groupby(['College', 'Year'])[['Base Pay']].mean().sort_values(by = 'Year').plot(kind = 'bar').plot()
    plt.title(coll[i], fontsize = 16)
    plt.ylabel("Average Base Pay")
plt.tight_layout
plt.show()

In [15]:
df1['Primary Job Title'].value_counts().head()

In [16]:
[x for x in df1['Primary Job Title'].unique() if 'research' in x.lower()]

In [36]:
x = [x for x in df1['Primary Job Title'].unique() if 'lecturer' in x.lower()]
df1.loc[df1['Primary Job Title'].isin(x)].groupby('Primary Job Title')[['Base Pay']].mean()

In [17]:
[x for x in df1['Primary Job Title'].unique() if 'lecturer' in x.lower()]

In [18]:
colleges = df3.loc[df3['Undergraduate College'] == 'Yes', 'College Abbreviation']
colleges

In [19]:
df3.loc[df3['College Abbreviation'] == 'LCOMEO', 'Notes']

In [20]:
df1

In [21]:
df1['Department'].value_counts()

In [37]:
df1['College'].value_counts()